### Emily Yamauchi
#### HW 1

## Exercise 2

1. First, download the data. We will again use the `penguins` dataset.

In [1]:
#setup

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model

file = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv'
penguins = pd.read_csv(file, sep=',', header=0)

In [2]:
penguins.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


2. Consider a regression problem in which we wish to predict a penguin’s body mass (i.e. the `body_mass_g` feature) from the features `bill_length_mm`, `bill_depth_mm`, and `flipper_length_mm`. Define the features $X$ and regression target $Y$ accordingly. What are their dimensions?

In [3]:
Y = penguins[['body_mass_g']].dropna()
Y.shape

(342, 1)

In [4]:
X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']].dropna()
X.shape

(342, 3)

3. Split $X$ and $Y$ into training and test sets using an 80-20 train/test split.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size = 0.2,
    random_state = 0)

X_train.shape, X_test.shape

((273, 3), (69, 3))

4. Instantiate and fit scikit-learn's `Ridge` model on the train data, with $\lambda$=1.0. Note that scikit-learn uses the notation `alpha` for what we call $\lambda$.

In [6]:
reg = linear_model.Ridge(alpha = 1.0)
reg.fit(X_train, y_train)

Ridge()

5. We will use the *mean squared error* (MSE) as a measure of model performance. Given a vector of model predictions $\hat{Y} \in\mathbb{R}^n$ and the true corresponding target values *Y*, the MSE is computed as  

$$
MSE(\hat{Y},Y) = \frac{1}{n}\left\|\hat{Y}-Y\right\|_2^2
$$

Implement this function and compute the MSE of the model from step (3) on the training set.

In [7]:
def mean_sq_err(y_hat, y):
    """
    Computes the squared errors of the predicted and actual value, divided by n
    Returns MSE value
    """
    
    n = len(y)
    
    sum_sq = np.sum(np.square(y_hat - y))
        
    return sum_sq / n

In [8]:
y_hat = np.mean(Y.values)

In [9]:
mean_sq_err(y_hat, y_test.values)

572136.7862352297

6. Now use five-by-two validation to select a value for $\lambda$ from set of values in the following array:  

`lam_vals = np.logspace(-2, 4 ,19)`  

The numpy function `logspace()` produces a sequence of logarithmically-spaced values

In [10]:
lam_vals = np.logspace(-2, 4, 19)

In [11]:
def five_by_two_validation(X_data, Y_data, lamb):
    
    """
    Perform a five-by-two validation on the ridge model
    For each value of your hyperparameter, repeat the following procedure five times:
    -Shuffle the indices of the training data.
    -Split the training data evenly into two parts. Call these the "train" and "validation" sets.
    -Train the model using data from the "train" set.
    -Generate model predictions for the "validation" set and record the prediction accuracy.
    -Swap the role of the "train" and "validation" sets and repeat the training and prediction steps above.
    Select the hyperparameter value with the best average prediction accuracy over all 10 trials.
    """
    
    n = len(X_data)
    nrow = int(np.floor(n/2)) # number of rows for splitting the dataset
    acc_df = pd.DataFrame(columns = lamb)

    
    for j in range(len(lamb)):
        acc_list = []
    
        for i in range(5): 

            #Shuffle index
            x_shuffle = X_data.sample(frac = 1)
            y_shuffle = Y_data.sample(frac = 1)

            #Split training data
            x_train = x_shuffle.iloc[0:nrow]
            y_train = y_shuffle.iloc[0:nrow]
            x_val = x_shuffle.iloc[nrow:n]
            y_val = y_shuffle.iloc[nrow:n]

            #Train the model using data from the "train" set
            model = linear_model.Ridge(alpha = lamb[j]).fit(x_train, y_train)

            #Generate model predictions for the "validation" set and record the prediction accuracy
            pred = model.predict(x_val)
            acc_list.append(mean_sq_err(np.mean(y_train.values), pred))

            #Swap the role of "train" and "validation" sets
            model_swap = linear_model.Ridge(alpha = lamb[j]).fit(x_val, y_val)
            pred_swap = model_swap.predict(x_train)
            acc_list.append(mean_sq_err(np.mean(y_val.values), pred_swap))
            
        acc_df[lamb[j]] = acc_list
        #res_df = acc_df.transpose().reset_index().rename(columns = {'index':'lambda'})
        
    return acc_df
        

In [12]:
lam_df = five_by_two_validation(X_train, y_train, lam_vals)
lam_df.head()

,0.010000,0.021544,0.046416,0.100000,0.215443,0.464159,1.000000,2.154435,4.641589,10.000000,21.544347,46.415888,100.000000,215.443469,464.158883,1000.000000,2154.434690,4641.588834,10000.000000
0,9671.061633,28510.075496,32131.467414,5999.575819,12955.602506,58731.025845,11420.705442,1580.036352,11776.440960,7669.343297,4923.136079,7866.737572,3385.701484,3951.878437,4318.928331,13970.687539,3915.985915,1346.255739,1451.299468
1,14321.964207,7654.662034,17527.127383,26729.228388,9565.870815,8004.321596,228.863935,21376.717384,13682.983050,1222.286262,2746.780692,58250.318251,40867.181328,19532.770354,7772.177798,3814.350528,2171.376770,9665.484812,2243.932269
2,8974.441351,20238.539938,24966.559776,2039.742764,3569.143039,8216.736626,8543.030672,33784.082333,6429.293083,6376.273446,2687.885657,6318.990622,8786.949449,30321.360646,26851.836091,13661.057314,8336.150634,6240.277576,2357.391580
3,12210.861278,7907.349026,33049.554684,10797.404117,3661.788139,6629.884473,5000.439778,1739.507893,16897.026485,11152.818859,11147.013103,14988.162135,4634.397845,20079.238609,5786.529842,7449.245727,16701.502411,3945.750130,2433.854277
4,21179.261561,17879.277016,15073.422289,18974.807102,3496.001070,20533.636711,16424.289173,4308.850561,31313.951136,29206.309865,8548.639297,3332.863317,15509.053864,6552.396389,4451.834981,11685.347572,5393.660568,1734.390036,17896.780798


7. Report the best-performing choice of $\lambda^*$ according to the validation procedure.

In [13]:
def get_min_lambda(X_data, y_data, lamb):
    """
    Perform 5x2 validation on training set
    Return the lambda value that results in best performing choice
    """
    
    lam_df = five_by_two_validation(X_data, y_data, lamb)
    min_index = np.argmin(lam_df.min())
    
    return lam_df.columns[min_index]

In [14]:
get_min_lambda(X_train, y_train, lam_vals)

4641.588833612773

8. Fit a ridge regression model to the entire training data using this choice of regularization parameter.

In [15]:
lam_best = get_min_lambda(X_train, y_train, lam_vals)
reg_best = linear_model.Ridge(alpha = lam_best).fit(X_train, y_train)
pred_best = reg_best.predict(X_test)

9. Compute and report the MSE of the above model on the test set

In [16]:
mean_sq_err(np.mean(y_test.values), pred_best)

334875.6827939941